1. 찾고 있는 장르
2. 영상 길이
3. 추천 타임라인

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/movies_metadata.csv')
df.columns

/home/kok/anaconda3/envs/rjs/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [3]:
#  필요한 열만 추출
df = df[['title', 'genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]

df.head()

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [8]:
# datetime 포멧으로 변경
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# 연 추출
df['year'] = df['release_date'].dt.year

df.head()

,title,genres,release_date,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0,1995.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0,1995.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0,1995.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0,1995.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0,1995.0


In [9]:
# 열 드롭
df = df.drop('release_date', axis=1)
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995.0


## Genres
- json 타입
- literal_eval 메서드를 사용해 파이썬 형식으로 바꿀 수 있음

In [10]:
# 첫 번째 영화의 장르
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [11]:
from ast import literal_eval

a = "[1, 2, 3]"
print(type(a))

b = literal_eval(a)
print(type(b))

<class 'str'>
<class 'list'>


In [12]:
# 결측치 -> 빈 리스트 [] 로 변경
df['genres'] = df['genres'].fillna('[]')

# str -> list
df['genres'] = df['genres'].apply(literal_eval)

# 딕셔너리 리스트를 문자열을 담은 리스트로 변환
df['genres'] = df['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[Animation, Comedy, Family]",81.0,7.7,5415.0,1995.0
1,Jumanji,"[Adventure, Fantasy, Family]",104.0,6.9,2413.0,1995.0
2,Grumpier Old Men,"[Romance, Comedy]",101.0,6.5,92.0,1995.0
3,Waiting to Exhale,"[Comedy, Drama, Romance]",127.0,6.1,34.0,1995.0
4,Father of the Bride Part II,[Comedy],106.0,5.7,173.0,1995.0


In [13]:
# 장르 여러개 -> 여러 행으로 펼치기
s = df.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'

gen_df = df.drop('genres', axis=1).join(s)
gen_df.head()

/tmp/ipykernel_997409/2025051304.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)


,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995.0,Animation
0,Toy Story,81.0,7.7,5415.0,1995.0,Comedy
0,Toy Story,81.0,7.7,5415.0,1995.0,Family
1,Jumanji,104.0,6.9,2413.0,1995.0,Adventure
1,Jumanji,104.0,6.9,2413.0,1995.0,Fantasy


## The built_chart function
1. 사용자 선호 가져오기
2. 유저와 맞는 모든 영화 추출
3. 해당 영화의 m, C 계산

In [14]:
def build_chart(gen_df, percentile=0.8):
    # 선호 장르 요청
    print('Input preferred genre')
    genre = input()
    # 제일 짧은 영상 길이?
    print('Input shortest duraction')
    low_time = int(input())
    # 가장 긴 영상 길이?
    print('Input longest duration')
    high_time = int(input())
    # 옛날 영화 bound
    print('Input latest earliest year')
    low_year = int(input())
    # 최신 영화 bound
    print('Input latest year')
    high_year = int(input())

    # 선호 영화 저장
    movies = gen_df.copy()
    movies = movies[(movies['genre'] == genre) &
                    (movies['runtime'] >= low_time) &
                    (movies['runtime'] < high_time) &
                    (movies['year'] >= low_year) &
                    (movies['year'] < high_year)]

    # m, C 계산
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)

    # m 이상의 투표 받은 영화 저장
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    # IMDB 스코어 계산
    q_movies['score'] = q_movies.apply(lambda x:
    (x['vote_count'] / (x['vote_count'] + m) * x['vote_average']) + (m / (m + x['vote_count']) * C), axis=1)

    # 스코어 내림차순 정렬
    q_movies = q_movies.sort_values('score', ascending=False)
    return q_movies

In [15]:
build_chart(gen_df).head()

Input preferred genre
Input shortest duraction
Input longest duration
Input latest earliest year
Input latest year


ValueError: Wrong number of items passed 6, placement implies 1